# Programming Assignment 7. Exploring Grover's Search Algorithm

In this assignment, you will explore the behavior of Grover's search algorithm on problem instances of increasing size. The problem we'll consider is finding bit strings that consisting of alternating bit pairs.

In [1]:
import matplotlib.pyplot as plt
import qsharp

Preparing Q# environment...


## Task 1. Implement the phase oracle (2 points)
First we need to define an oracle. 
For this example we will consider the function $f(x)$ which is equal to 1 when $x$ is a bit string of length $N$ in which odd pairs of adjacent bits are the same and even pairs differ. For any $N$ there will be two possible solutions, one starting with 0 and one starting with 1. So for this example the number of solutions is 2.

Our $f$ will look like this:
$$
f(x) =  \begin{cases} 
      1 & x = 00110011\text{... or }11001100\text{...}\\
      0 & \text{otherwise}
   \end{cases}
$$

In [ ]:
%%qsharp
operation AlternatingBitPairsOracle(qs : Qubit[]) : Unit {
    // Task 1: implement the phase oracle for this algorithm
    // ...
}

You can use the following wrapper operation to print the effects your oracle has on the even superposition of basis states. 

**IMPORTANT**: depending on the version of `qsharp` package you're using, the output of `DumpMachine` will differ. Versions 1.0.34 and earlier reverse the order of qubits in the output, so if you run an oracle that correctly flips the phase of states $|110\rangle$ and $|001\rangle$ for a three-qubit state, DumpMachine will show them as $|011\rangle$ and $|100\rangle$. Versions 1.1 and later don't reverse the order of qubits, so the output will be as expected. You can check which version you're using from the notebook, by runinng `!pip list` in a separate cell and looking for `qsharp` package in the output.

In [ ]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

// The operation that applies the oracle to an even superposition of all inputs and prints the resulting state
operation DemoWrapper(N : Int) : Unit {
    use qs = Qubit[N];
    ApplyToEach(H, qs);
    AlternatingBitPairsOracle(qs);
    DumpMachine();
    ResetAll(qs);
}


In [ ]:
qsharp.eval("DemoWrapper(5)")

## Task 2. How far can you optimize your solution? (3 points)

You can use the code below to estimate the resources required to run your solution. The scoring is done based on the case of $N = 5$.

* If your solution gets a score above 1500, you'll get 0 points.
* If your solution gets a score between 1000 and 1500, you'll get 1 point.
* If your solution gets a score between 500 and 1000, you'll get 2 points.
* If your solution gets a score under 500, you'll get 3 points.

In [ ]:
%%qsharp
open Microsoft.Quantum.Measurement;
operation AlternatingBitPairsOracleWrapper(N : Int) : Result[] {
    use qs = Qubit[N];
    AlternatingBitPairsOracle(qs);
    return MeasureEachZ(qs);
}

In [ ]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth

est = qsharp.estimate("AlternatingBitPairsOracleWrapper(5)")
evaluate_results(est)

### Task 3. Find the optimal number of iterations for $N = 3, 4, 5$ (2 points)

The next cell implements Grover's search algorithm that takes two parameters: the number of bits in the strings we're looking for, and the number of iterations to run the algorithm for before measurement. 

Your task is to calculate the optimal number of iterations for $N = 3, 4, 5$ based on the size of the search space and the known number of problem solutions.

In [ ]:
%%qsharp
open Microsoft.Quantum.Measurement;
operation Grovers(N : Int, iterations : Int) : Result[] {
    use qs = Qubit[N];
    ApplyToEach(H, qs);

    for i in 1 .. iterations {
        AlternatingBitPairsOracle(qs);
        
        within {
            ApplyToEachA(H, qs);
            ApplyToEachA(X, qs);
        } apply {
            Controlled Z(qs[...N - 2], qs[N - 1]);
        }
    }
    let res = MeasureEachZ(qs);
    ResetAll(qs);
    return res;
}

You can run a single simulation to see whether the measurement produced a state that is a solution to our problem. 

In [ ]:
qsharp.eval("Grovers(4, 1)")

You can also run a series of simulations on a local simulator to see the percentage of shots for which a query returns a correct answer.

In [ ]:
N = 4
iterations = 1

target1 = [qsharp.Result.One if (i // 2) % 2 == 1 else qsharp.Result.Zero for i in range(N)]
target2 = [qsharp.Result.One if (i // 2) % 2 == 1 else qsharp.Result.Zero for i in range(2, N+2)]
shots = 100
correctCount = 0
results = qsharp.run(f"Grovers({N}, {iterations})", shots=shots)

for res in results:
    if res == target1 or res == target2:
        correctCount += 1

print(f"N={N}, iter={iterations} - {100 * correctCount/shots}% of queries returned a correct answer")


## Tasks 4-5. Run Grover's search for $N = 3, 4, 5$ on noiseless Rigetti simulator (1 point) and noisy Quantinuum emulator (2 points)

* Run the Grover's search algorithm for the oracle you've implemented on Rigetti or IonQ noiseless simulator (`rigetti.sim.qvm` or `ionq.simulator`) for $N = 3, 4, 5$.
* Run the Grover's search algorithm for the oracle you've implemented on Quantinuum noisy emulator (`quantinuum.sim.h1-1e`) for $N = 3, 4, 5$.
* Plot the results in a 2x3 or 3x2 table to compare the results on the same problem instance side-by-side.

In [ ]:
from azure.quantum import Workspace

workspace = Workspace(
      resource_id = "/subscriptions/006c31bd-51e0-41db-b622-d1d3f8e7213a/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/AQ-Demo",
      location = "westus")
qsharp.init(target_profile=qsharp.TargetProfile.Base)

**IMPORTANT:** After you call `qsharp.init`, you'll need to recompile the cells that define the oracle and Grover's search before proceeding.

In [ ]:
# Select the target here
target = workspace.get_targets("ionq.simulator")

In [ ]:
N = 4
iterations = 1

program = qsharp.compile(f"Grovers({N}, {iterations})")
job = target.submit(program, f"Alternating bit pairs (Grover): N={N}, iter={iterations}", shots=1000)

In [ ]:
# Once the job is complete, you can fetch its results and print them manually
res = job.get_results()
res

The cells below allow you to fetch job results based on its ID and plot them, highlighting the results that correspond to the correct answers.

In [ ]:
def plot_job_results(jobId, iter, sim):
    # Fetch job results for the given jobId and plot them.
    # "iter" and "sim" parameters are used to make the plot header more descriptive
    job = workspace.get_job(jobId)
    res = job.get_results()
    print(res)

    # Get keys and values, and convert keys from lists of bits to bitstrings
    keys, outputFreq = zip(*sorted(res.items()))
    keys = [key[1:-1].replace(", ", "") for key in keys]
    keyLen = len(keys[0])

    # Generate correct bitstrings
    firstBitString = ""
    secondBitString = ""
    for i in range(keyLen):
        firstBitString += str((i // 2) % 2)
        secondBitString += str(1 - (i // 2) % 2)

    # Find the indices of correct answers in the array of keys
    firstBitStringLoc = min(keys.index(firstBitString), keys.index(secondBitString))
    secondBitStringLoc = max(keys.index(firstBitString), keys.index(secondBitString))

    # Plot the frequencies of all keys, with red color marking correct answers and blue - incorrect ones
    plt.bar(keys[0:firstBitStringLoc], outputFreq[0:firstBitStringLoc], color = "blue")
    plt.bar(keys[firstBitStringLoc], outputFreq[firstBitStringLoc], color = "red")
    plt.bar(keys[firstBitStringLoc+1:secondBitStringLoc], outputFreq[firstBitStringLoc+1:secondBitStringLoc], color = "blue")
    plt.bar(keys[secondBitStringLoc], outputFreq[secondBitStringLoc], color = "red")
    plt.bar(keys[secondBitStringLoc+1:], outputFreq[secondBitStringLoc+1:], color = "blue")
    plt.xticks(rotation=90)
    plt.title("{} results for N = {} iter = {}".format(sim, keyLen, iter))
    plt.show()

In [ ]:
plot_job_results("<job ID>", iterations, "<target name>")